# What Can You Expect In this Notebook?

## Goals
1. Create a collaborative filtering feature
1. Make book clustering

In [8]:
# import libraries (you may add additional imports but you may not have to)
import logging
logging.captureWarnings(True)

import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)
from scipy.sparse import csr_matrix

#viz lib
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import seaborn as sns

#to scale the data using z-score
from sklearn.preprocessing import StandardScaler

#Importing PCA
from sklearn.decomposition import PCA

In [5]:
df = pd.read_csv("data/clean_data.csv")

In [6]:
df.head()

,Unnamed: 0,isbn,book_title,book_author,year_of_publication,publisher,mod_title,user_id,book_rating,location,age
0,0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,classical mythology,2.0,0.0,"stockton, california, usa",18.0
1,1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,clara callan,8.0,5.0,"timmins, ontario, canada",24.0
2,2,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,clara callan,11400.0,0.0,"ottawa, ontario, canada",49.0
3,3,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,clara callan,11676.0,8.0,"n/a, n/a, n/a",24.0
4,4,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,clara callan,41385.0,0.0,"sudbury, ontario, canada",24.0


## Advice from freeCodeCamp
https://www.freecodecamp.org/learn/machine-learning-with-python/machine-learning-with-python-projects/book-recommendation-engine-using-knn<br>
If you graph the dataset (optional), you will notice that most books are not rated frequently. To ensure statistical significance, remove from the dataset users with less than 200 ratings and books with less than 100 ratings.

# Find how many books have each users read
user_stat = pd.pivot_table(
    data = df,
    values = 'book_title',
    index = 'user_id',
    aggfunc= 'count'
).reset_index()